# Modélisation

In [1]:
# Bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
import math
import operator

def RMSLE(estimate, real):
    """
        Computes the Root Mean Square Logarithmic Error of two elements.
        
        :param estimate: The predicted values
        :param real: The real values
        :return: The RMSLE of the two input elements
    """
    
    # we check the size on the input elements are the same
    assert(estimate.size == real.size), "Input elements must have the same size"
    size = estimate.size
    diff = map(operator.sub, 
               map(lambda x:math.log(x+1), estimate),
               map(lambda x:math.log(x+1), real))
    diff_squared = map(lambda x:x*x, diff)
    return math.sqrt(sum(diff_squared)/size)

In [2]:
# Chemin vers le fichier csv contenant les données à explorer.
path= "data/cleaned/mower_market_snapshot_cleaned.csv"

# Délimiteur utilisé dans le fichier csv
delimiter=","

# Création de la dataframe à partir du fichier csv
mower_df = pd.read_csv(path, delimiter=delimiter)

In [3]:
data = mower_df.ix[:, mower_df.columns != 'attractiveness']
data.drop(labels="id", axis=1, inplace=True)
target = mower_df["attractiveness"]

# we remove 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.2)

/Users/mathieu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## Support Vector Classification with linear kernel

In [4]:
clf_svr_lin = SVR(kernel='linear', tol=0.001)
# training
clf_svr_lin = clf_svr_lin.fit(X_train.values, y_train.values)
# testing
result_svr_lin = clf_svr_lin.predict(X_test.values)
# error
RMSLE(result_svr_lin, y_test.values)

0.07072015253136421

## Support Vector Classification

In [5]:
reg = linear_model.Ridge(alpha = .5)
# training
reg.fit(X_train.values, y_train.values)
# testing
result_reg = reg.predict(X_test.values)
# error
RMSLE(result_reg, y_test.values)

0.06976873200261322

## Support Vector Classification with rbf kernel

In [6]:
clf_rbf = SVR(kernel='rbf')
clf_rbf = clf_rbf.fit(X_train.values, y_train.values)
result_rbf = clf_rbf.predict(X_test.values)
RMSLE(result_rbf, y_test.values)

0.05512953004084889

## Random forest

In [7]:
clf_rfg = RandomForestRegressor(100)
clf_rfg = clf_rfg.fit(X_train.values, y_train.values)
result_rfg = clf_rfg.predict(X_test.values)
RMSLE(result_rfg, y_test.values)

0.04436744978939558

La fôret aléatoire semble le meilleur candidat pour réaliser notre prédiction

## Prédiction

In [8]:
# Chemin vers le fichier csv contenant les données à étudier.
path= "data/original/submission_set.csv"

# Délimiteur utilisé dans le fichier csv
delimiter_submission=";"

# Création de la dataframe à partir du fichier csv
submission = pd.read_csv(path, delimiter=delimiter_submission)

# On supprime les variables que nous ne voulons pas garder pour la prédiction
submission.drop(labels="margin", axis=1, inplace=True)
submission.drop(labels="prod_cost", axis=1, inplace=True)
submission.drop(labels="warranty", axis=1, inplace=True)

# binarisation des variables catégorielles
submission_df = pd.get_dummies(submission)

In [9]:
submission_df.describe(include='all')

,capacity,failure_rate,id,price,product_type_auto-portee,product_type_electrique,product_type_essence,quality_Hight,quality_Low,quality_Medium
count,600.000000,600.000000,600.000000,600.000000,600.00000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,50.101143,0.164916,19957.500000,244.531361,0.10000,0.308333,0.591667,0.123333,0.678333,0.198333
std,16.364164,0.073387,173.349358,241.642303,0.30025,0.462190,0.491936,0.329094,0.467506,0.399077
min,7.432330,0.029201,19658.000000,63.969204,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,38.754969,0.095825,19807.750000,73.367481,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50.761661,0.202148,19957.500000,78.618777,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,61.342713,0.225939,20107.250000,380.587182,0.00000,1.000000,1.000000,0.000000,1.000000,0.000000
max,92.666666,0.280646,20257.000000,919.588021,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# On garde les ids pour enregistrer notre prédiction
ids = submission_df["id"]
submission_df.drop(labels="id", axis=1, inplace=True)

In [11]:
# prédiction en utilisant le modèle donné par la fôret aléatoire
submission_result = clf_rfg.predict(submission_df.values)

col = ['id', 'attractiveness']
submission_to_save = pd.DataFrame(zip(ids,submission_result), columns=col)

In [12]:
# On enregistre au format csv
submission_to_save.to_csv(path_or_buf="data/result/besson_mathieu_attractiveness.csv", sep=';', index=False)

In [13]:
submission_to_save.describe()

,id,attractiveness
count,600.000000,600.000000
mean,19957.500000,0.641109
std,173.349358,0.112259
min,19658.000000,0.353832
25%,19807.750000,0.557622
50%,19957.500000,0.643397
75%,20107.250000,0.725564
max,20257.000000,0.878987
